In [ ]:
from pathlib import Path
import json
import xarray as xr

In [ ]:
default_data_vars = json.load(open("run_scripts/new_vars.json"))["default"]

In [ ]:
scratch = Path("/lcrc/group/e3sm/ac.mkelleher/scratch/chrys/")
# case = "20221130.F2010.ne4_oQU240.dtcl_control_n0030"
# case = "20221201.F2010.ne4_oQU240.dtcl_zmconv_c0_0p00201_n0030"
case = "20221206.F2010.ne4_oQU240.dtcl_zmconv_c0_0p0030_n0030"
case_dir = Path(scratch, case, "run")
ninst = 120
files = {
    inst: sorted(case_dir.glob(f"{case}.eam_{inst:04d}*aavg.nc"))
    for inst in range(1, ninst + 1)
}

In [ ]:
data_1 = xr.open_dataset(files[1][0])
data_110 = xr.open_dataset(files[110][0])
common_vars = set(default_data_vars).intersection(data.data_vars)
print(f"N default: {len(default_data_vars)}\nN   input: {len(data.data_vars)}\nN  common: {len(common_vars)}")

In [ ]:
def check_ensemble_vars(ens_data, common_vars):
    """Check all common_vars to make sure they're not constant across the ensemble."""
    const_vars = []
    for _var in common_vars:
        ens_std = ens_data[_var].mean(dim="time").std(dim="ens").values
        if ens_std == 0.0:
            const_vars.append(_var)
    return const_vars

ens_data = xr.open_mfdataset([files[inst][0] for inst in files], combine="nested", concat_dim="ens")
const_vars = check_ensemble_vars(ens_data, common_vars)

In [ ]:
for _var in const_vars:
    common_vars.remove(_var)

In [ ]:
out_vars = {"default": sorted(list(common_vars))}
with open(Path("./run_scripts", "new_vars.json"), "w") as _fout:
    _fout.write(json.dumps(out_vars))

In [ ]:
set(data_1.data_vars).difference(data_110.data_vars)